<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian-Deep-learning-with-Discreet-wavelet/blob/master/Copy_of_Dissertation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pywt
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load your datasets (assuming df1 contains the sensor values, df2 contains the ground truth)
df1 = pd.read_csv('/content/drive/My Drive/Sensor1_values_bias_0_1_dur_10_dep.csv').astype(np.float32)
df2 = pd.read_csv('/content/drive/My Drive/Ground_truth_bias_0_1_dur_10_sensor1_dep.csv')

# Assuming df2 contains the ground truth (binary classification, 0 or 1)
y = df2['ground_truth'].values  # Replace with the correct column name for ground truth
X = df1.values  # Sensor readings as feature matrix

# Discrete Wavelet Transform (DWT) for feature extraction
def perform_dwt(data, wavelet='db4', level=3):
    """
    Apply Discrete Wavelet Transform to the input data.
    :param data: 1D signal array.
    :param wavelet: Wavelet type (default: 'db4').
    :param level: Decomposition level (default: 3).
    :return: Concatenated wavelet coefficients as features.
    """
    coeffs = pywt.wavedec(data, wavelet, level=level)
    # Flatten the wavelet coefficients for input to NN
    return np.hstack(coeffs)

# Bayesian Dense Layer (Custom)
class BayesianDenseLayer(tf.keras.layers.Layer):
    """
    Custom Bayesian Dense layer using a normal distribution over the weights.
    """
    def __init__(self, units, prior_fn, posterior_fn, activation=None, **kwargs):
        super(BayesianDenseLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.prior_fn = prior_fn
        self.posterior_fn = posterior_fn

    def build(self, input_shape):
        # Initialize the weights and biases
        self.w = self.add_weight(
            name="weights",
            shape=(input_shape[1], self.units),
            initializer="random_normal",
            regularizer=None,
            trainable=True
        )
        self.b = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="zeros",
            regularizer=None,
            trainable=True
        )
        super(BayesianDenseLayer, self).build(input_shape)

    def call(self, inputs):
        # Compute the output using the learned weights
        z = tf.matmul(inputs, self.w) + self.b
        if self.activation:
            z = self.activation(z)
        return z

# Helper functions for prior and posterior distributions
def make_prior_fn():
    """Define the prior distribution as Normal."""
    tfd = tfp.distributions
    return tfd.Normal(loc=0., scale=1.)

def make_posterior_fn():
    """Define the posterior distribution as Normal with a softplus scale."""
    tfd = tfp.distributions
    return lambda params: tfd.Normal(loc=params[..., :1], scale=tf.nn.softplus(params[..., 1:]))

# Bayesian Neural Network Model
class BayesianNN(Model):
    """
    Custom Bayesian Neural Network that incorporates uncertainty over weights.
    """
    def __init__(self, input_shape, prior_fn, posterior_fn):
        super(BayesianNN, self).__init__()
        self.dense1 = BayesianDenseLayer(128, prior_fn, posterior_fn, activation=tf.nn.relu)
        self.dense2 = BayesianDenseLayer(64, prior_fn, posterior_fn, activation=tf.nn.relu)
        self.dense3 = BayesianDenseLayer(32, prior_fn, posterior_fn, activation=tf.nn.relu)
        self.dense4 = BayesianDenseLayer(16, prior_fn, posterior_fn, activation=tf.nn.relu)
        self.dense5 = BayesianDenseLayer(8, prior_fn, posterior_fn, activation=tf.nn.relu)
        self.output_layer = BayesianDenseLayer(1, prior_fn, posterior_fn, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return self.output_layer(x)

# Data Preprocessing and DWT Extraction
def preprocess_data(X, y, wavelet='db4', level=3):
    """
    Apply DWT to input data and preprocess it for use in Bayesian Neural Network.
    """
    # Apply DWT to each sample in the dataset
    X_dwt = np.array([perform_dwt(x, wavelet, level) for x in X])

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_dwt)

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Model Training and Evaluation
def train_and_evaluate():
    # Step 1: Generate and preprocess data
    X_train, X_test, y_train, y_test = preprocess_data(X, y)

    # Step 2: Build the Bayesian Neural Network
    input_shape = X_train.shape[1:]
    prior_fn = make_prior_fn()
    posterior_fn = make_posterior_fn()
    model = BayesianNN(input_shape, prior_fn, posterior_fn)

    # Compile the model with a loss function and optimizer
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Step 3: Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

    # Step 4: Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

    # Step 5: Predict and analyze uncertainty
    predictions = model(X_test)
    print("Predictions:", predictions.numpy())

    # Plot the predicted values vs actual values
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='True Values', alpha=0.7)
    plt.plot(predictions.numpy(), label='Predicted Values', alpha=0.7)
    plt.legend()
    plt.title("True vs Predicted Values")
    plt.show()

if __name__ == "__main__":
    train_and_evaluate()


Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://ea464da4.ngrok.io
